## Problem Set 2


### Problem 1

Clean up the file. This means getting rid of duplicates; you can assume that no student can register for the same course more than once. How many duplicate records do you find? Some of the fields have bad or missing values; repair those that you can (and explain what a repair means).

### Import Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import hashlib
from functools import reduce

In [12]:
# NA_FILL_VALUE will replace missing values in the data with 0
NA_FILL_VALUE = 0

In [13]:
# Get the total number of rows in the data set prior to filtering out bad, missing, or corrupt lines
# use the number to compare the size of the data set after filtering 
columnNames = []
with open('dirty_sample_small_header.csv', 'r') as headerFile:
    headerReader = csv.reader(headerFile, delimiter=',')
    for row in headerReader:
        columnNames.append(row[1])
        
numCols = len(columnNames)

Upon initial inspection, the "ip" column contains values do not match the format of an ip address. Each column after ip also has incompatible row entires. The "viewed" values must be missing under its appropriate column header because it is logically infeasible to have viewed, explored, or completed a course without registering for it, or for exploring a course without viewing it. As a result of the shift, the last column, the former cert_status values are eliminated. This is reasonable because the numbers previously under cert_status resembled times, and the values previously to the left column of cert_status were statuses ie: "downloadable" 

In [4]:
invalidCols = 0; duplicateRows = 0; keptRows = 0; missingCols = 0; totalRows = 0
onHeader = True
hashes = set()
with open('dirty_sample_small.csv', 'r') as dataFile:
    with open('valid_rows_sample_small.csv', 'w') as outFile:
        dataReader = csv.reader(dataFile, delimiter=',')
        outWriter = csv.writer(outFile, delimiter = ',')
        for row in dataReader:
            # Skip the header line
            if onHeader:
                # Because the column "viewed" has data that are logically inconsistent, drop it.  
                # Some entries have "Registered=True" and "Viewed=False". The values seem to actually be more similar
                # to "explored" than viewed.  Additionally including one of viewed, explored, certified, or completed
                # makes the column headers not align with the data so we need to drop one of them. 
                row = list(filter(lambda x: x != "viewed", row))
                outWriter.writerow(row)
                onHeader = False; continue
        
            totalRows += 1
            # Ignore rows with incorrect number of columns
            if len(row) != numCols:
                invalidCols += 1
                continue 
            # Get the md5 hash of each row to determine whether the row is duplicated or not
            else:
                m = hashlib.md5()
                
                # Two rows are identical if they have the same (course_id, student_id)
                # fullRowStr = reduce((lambda x, y: x + y), row).encode('utf-8')
                rowStr = (row[0] + row[1]).encode('utf-8')
                m.update(rowStr)
                hashedRow = m.hexdigest()
                if hashedRow in hashes:
                    duplicateRows += 1
                    continue 
                # If it's a new row, write it to the cleaned dataset valid_rows...
                else:
                    hashes.add(hashedRow)
                    # Ignore rows where more than 1/2 of the entries are missing
                    # Count the number of nan's in a row
                    missingFields = reduce(lambda x, y: x + int(y == ""), row, 0) # do not change "" to ''
                    
                    if missingFields >= int(0.5 * numCols):
                        missingCols += 1
                        continue
                    else:
                        keptRows += 1
                        # Also ignore the data in the very last column as it does not seem to correspond to any of the 
                        # columns in this area of the dataset
                        outWriter.writerow(row[:-1])
print("Dropped: %d   Duplicates: %d   Missing: %d   Kept: %d   Total: %d" % (invalidCols, duplicateRows, missingCols,
                                                                             keptRows, totalRows))

# If we only drop duplicates that match on all fields these are the results.   
# Dropped: 15708   Duplicates: 595797   Kept: 49981   Total: 661486

Dropped: 15708   Duplicates: 595797   Missing: 6275   Kept: 43706   Total: 661486


After removing lines that were corrupted (they had a number of entries that differed from the number specified by the header file), we found that there were 595797 duplicate rows. The remaining 49981 rows, while not perfect, seem to be have empty fields for roughly the same number of columns – if we keep rows that have a value in at least 55% of their fields then we keep all the de-duplicated rows, but if this theshold is raised to 60% we drop all of those – suggesting that some of these fields may not be very useful for data analysis. For most of these missing values, there is not much we can do. However, for the count variables (the one's like "n &lt; noun &gt;") we set missing values to be 0 because we think it is unlikely that students would post and EdX would not count that.  Additionally for birthdays, which are the only user inputed field in this dataset, if the student provides a birthday that would indicate the student is older than 100 or younger than 12 we set that birthday to be the average (mean) birthday. The mean was calculated excluding bad birthdays. This change would reduce the variance of any estimator that uses the birthday but roughly preserves the mean birthday (because the replacement YoB is rounded to the nearest whole year).  



In [5]:
# Guarantees all rows are accounted for after filtering data
invalidCols + duplicateRows + missingCols + keptRows == totalRows

True

In order to repair bad or missing values, we must understand which columns these values come from, which type all the data in that column should be represented with, and how values ought to be be represented.

In [6]:
colToType = {
    "registered" : bool, 
    "explored" : bool,
    "certified" : bool,
    "completed" : bool,
    "latitude" : float, 
    "longitude" : float, 
    "YoB" : int, 
    "start_time" : "date",
    "first_event" : "date",
    "last_event" : "date", 
    "nevents" : int, 
    "ndays_act" : int, 
    "nplay_video" : int,
    "nchapters" : int, 
    "nforum_posts" : int, 
    "nforum_votes" : int, 
    "nforum_endorsed" : int, 
    "nforum_threads" : int, 
    "nforum_comments" : int, 
    "nforum_pinned" : int, 
    "nprogcheck" : int, 
    "nproblem_check" : int, 
    "nforum_events" : int, 
    # encoded as "0" or "1" (not "False" or "True"), need to convert to bool after converting to int
    "is_active" : int, 
    "cert_created_date" : "date", 
    "cert_modified_date" : "date"    
}

In [7]:
def convertToBool(x):
    if x == 'True': return True
    else: return False

In [8]:
df_test = pd.read_csv("valid_rows_sample_small.csv", sep=',', engine='python', error_bad_lines=False, dtype='unicode')

# Use Pandas drop_duplicates() as evidence that dataset is deduplicated
print("Deduplicated Valid Rows: %d\tFully Deduplicated: %r" 
      % (len(df_test), len(df_test) == len(df_test.drop_duplicates())))
print("Columns: %d" % len(df_test.columns.values))

# Convert types of columns
for colName, colType in colToType.items():
    if colType == "date":
        df_test[colName] = pd.to_datetime(df_test[colName])
    elif colType == int:
        df_test[colName] = df_test[colName].apply(lambda x: x if x != 'nan' else 0).astype(int)
    elif colType == float:
        df_test[colName] = df_test[colName].apply(lambda x: x if x != 'nan' else float('nan')).astype(float)
    elif colType == bool:
        df_test[colName] = df_test[colName].apply(lambda x: convertToBool(x))

# special case for is_active
df_test.is_active = df_test.is_active.astype(bool)

# Reset extreme birthdays to mean birthday
meanValidYoB = int(round(df_test[(df_test.YoB > 1920) & (df_test.YoB < 2008)].YoB.mean()))
df_test.loc[:, 'YoB'] = df_test.YoB.apply(lambda x: meanValidYoB if x > 0 and (x <= 1920 or x >= 2008) else x)


Deduplicated Valid Rows: 43706	Fully Deduplicated: True
Columns: 47


Some fields may have values that are incompatible types. This may occur when no data is stored for a variable, a user did not complete the course or course registration, or a column may contain multiple data types. A string representation of an age cannot be compared to a number. If a user inputted N/A, or left that field blank, it is interpreted differently as NA, na, NaN.

In [16]:
df_test.replace("nan", np.nan, inplace=True)
df_test.replace("None", np.nan, inplace=True)

### Problem 2

Some lines may be corrupt; get rid of those or mark them in some way to show that they are not good lines. How many corrupt lines are there? Does the count of corrupt lines change if you get rid of them before getting rid of the duplicate records? What difference might this make to the remaining data set?

Essentially the only difference between the code below and the code above is that instead of removing corrupt, incomplete lines before we remove duplicates, the first remove duplicates and then remove corrupt lines.  The count of corrupt lines did fall significantly (from 15708 to 498) once they're removed after the duplicates, but the total nnumber of records kept remained the same (49981). While in this specific dataset swapping these two procedures made produced identical cleaned datasets, in general these two operations may not commute, so to speak.  Because the lines are processed in the order in which they appear in the csv, if an incomplete row that also has the same (student_id, course_id) pair comes before a complete row witht the same id-paid, then in this case, we will completely eliminate this student from the dataset because the complete row(s) will be dropped by the deduplication and then the corrupt row will also be dropped afterwards.  Therefore, the order of filtering and processing in problem 1 is the better way because that will not allow a student such as this one to be removed.  
    

In [10]:
invalidCols = 0; duplicateRows = 0; keptRows = 0; missingCols = 0; totalRows = 0
onHeader = True
hashes = set()
with open('dirty_sample_small.csv', 'r') as dataFile:
    with open('valid_rows_sample_small_2.csv', 'w') as outFile:
        dataReader = csv.reader(dataFile, delimiter=',')
        outWriter = csv.writer(outFile, delimiter = ',')
        for row in dataReader:
            # Skip the header line
            if onHeader:
                # Because the column "viewed" has data that are logically inconsistent, drop it.  
                # Some entries have "Registered=True" and "Viewed=False". The values seem to actually be more similar
                # to "explored" than viewed.  Additionally including one of viewed, explored, certified, or completed
                # makes the column headers not align with the data so we need to drop one of them. 
                row = list(filter(lambda x: x != "viewed", row))
                outWriter.writerow(row)
                onHeader = False; continue
        
            totalRows += 1
            
            m = hashlib.md5()
                
            # Two rows are identical if they have the same (course_id, student_id)
            # fullRowStr = reduce((lambda x, y: x + y), row).encode('utf-8')
            rowStr = (row[0] + row[1]).encode('utf-8')
            m.update(rowStr)
            hashedRow = m.hexdigest()
            if hashedRow in hashes:
                duplicateRows += 1
                continue 
            # If it's a new row, write it to the cleaned dataset valid_rows...
            else:
                # Ignore rows with incorrect number of columns
                if len(row) != numCols:
                    invalidCols += 1
                    continue    
                
                # Ignore rows where more than 3/4 of the entries are missing
                # Count the number of nan's in a row
                missingFields = reduce(lambda x, y: x + int(y == ""), row, 0) # do not change "" to ''
                if missingFields >= int(0.45 * numCols):
                    missingCols += 1
                    continue
                else:
                    keptRows += 1
                    hashes.add(hashedRow)
                    # Also ignore the data in the very last column as it does not seem to correspond to any of the 
                    # columns in this area of the dataset
                    outWriter.writerow(row[:-1])
                        
print("Dropped: %d   Duplicates: %d   Missing: %d   Kept: %d   Total: %d" % (invalidCols, duplicateRows, missingCols,
                                                                             keptRows, totalRows))

Dropped: 3310   Duplicates: 506011   Missing: 110772   Kept: 41393   Total: 661486


### Problem 3

What are some possible sources of bias in this data set? Is there anything unusual about the data set that you should flag?

While the only user inputted data in this dataset were year of birth, gender, and level of education, generally, any data filled in by the user can introudce bias into the analysis because there is no way to verify that data and certain groups of people may not be willing to tell EdX their true characteristics, beliefs, or demographics.  Additionally, if people have more than one account at the same time, take the class multiple times, or if multiple use the same account, then our results could also be biased.  
    
It is unusual that when we load the original data in, the column headers to not seem to match the data entirely. For example, the "completed" field contains IP addresses, and more generally all of the data seems to have been shifted over by 1 column. This shifting was likely caused by losing the data that should have been in the "viewed" column.  Of all five boolean columns at the beginning (registered, viewed, explored, certified), the data in "viewed" are inconsistent with the other columns and more likely resemble what "explored" should contain.  Some entries have "Registered=True" and "Viewed=False". As a result, we removed "viewed" from our headers and also dropped the rightmost column in the dataset that was now without a header, but also doesn't seem to correspond to any other possible column. 
It was also unusual the last column of the data had data entries that did not seem to match the column header which was supposeduly indicating a certificate status. 